In [ ]:
!pip install numpy==1.26.4 rdkit-pypi

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
def cir_query(name, representation="smiles"):
    import requests
    url = f"https://cactus.nci.nih.gov/chemical/structure/{name}/{representation}"
    r = requests.get(url)
    return r.text if r.ok else None

smiles = cir_query("ibuprofen")
print(smiles)


In [ ]:
# Convert to RDKit molecule and draw
mol = Chem.MolFromSmiles(smiles)
Draw.MolToImage(mol)

In [ ]:
from rdkit.Chem import Descriptors

print("Molecular Weight:", Descriptors.MolWt(mol))
print("logP (Octanol/Water):", Descriptors.MolLogP(mol))
print("Number of H-bond donors:", Descriptors.NumHDonors(mol))
print("Number of H-bond acceptors:", Descriptors.NumHAcceptors(mol))
print("TPSA (Polar Surface Area):", Descriptors.TPSA(mol))
print("Rotatable Bonds:", Descriptors.NumRotatableBonds(mol))
print("Num Aromatic Rings:", Descriptors.NumAromaticRings(mol))


In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

import requests

# Helper function to get SMILES from CIR
def cir_query(name, representation="smiles"):
    url = f"https://cactus.nci.nih.gov/chemical/structure/{name}/{representation}"
    r = requests.get(url)
    return r.text if r.ok else None

# Define names
names = ["ibuprofen", "aspirin", "paracetamol", "caffeine", "naproxen", "cyclosporine"]

# Get molecules from names using CIR
smiles_list = [cir_query(name) for name in names]
mols = [Chem.MolFromSmiles(sm) if sm else None for sm in smiles_list]

# Filter out any None (failed lookups)
valid = [(name, mol) for name, mol in zip(names, mols) if mol is not None]
names, mols = zip(*valid)

# Generate Morgan fingerprints (ECFP4)
fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in mols]

# Similarity to ibuprofen (first entry)
similarities = [DataStructs.TanimotoSimilarity(fps[0], fp) for fp in fps]

# Show results
for name, sim in zip(names, similarities):
    print(f"{name.capitalize():<15}: similarity = {sim:.3f}")



In [ ]:
from rdkit.Chem import Draw

# Add similarity scores as legends
legends = [f"{name.capitalize()}\nSimilarity: {sim:.2f}" for name, sim in zip(names, similarities)]

# Display molecules in grid
Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(250,250), legends=legends)


In [ ]:
def passes_lipinski(mol):
    return (Descriptors.MolLogP(mol) < 5 and
            Descriptors.MolWt(mol) < 500 and
            Descriptors.NumHDonors(mol) <= 5 and
            Descriptors.NumHAcceptors(mol) <= 10)

for name, m in zip(names, mols):
    print(f"{name:<12} passes Lipinski? {passes_lipinski(m)}")


Cyclosporine is an immunosupressant medication and does not follow the rule of 5

In [ ]:
!pip install py3Dmol

In [ ]:
# 1. Load SMILES and add Hs
smiles = "C[C@@H](C1=CC=C(C=C1)CC(C)C)C(=O)[O-]"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# 2. Generate 3D coordinates
params = AllChem.ETKDGv3()
success = AllChem.EmbedMolecule(mol, params)
if success == 0:
    AllChem.UFFOptimizeMolecule(mol)
else:
    print("Embedding failed.")


In [ ]:
conf = mol.GetConformer()
xyz = f"{mol.GetNumAtoms()}\n\n"
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    xyz += f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}\n"
print(xyz)


In [ ]:
import py3Dmol

viewer = py3Dmol.view(width=400, height=300)
viewer.addModel(xyz, "xyz")
viewer.setStyle({"stick": {}})
viewer.setBackgroundColor("white")
viewer.zoomTo()
viewer.show()
